<a href="https://colab.research.google.com/github/JoeyBassilRD/LightweightAI-PROJECT/blob/main/xml_to_gen_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-**UPLOAD THE CHOSEN ANNOTATIONS.XML file for a specific shape**

 **Basic XML to Binary Mask (OpenCV Method)**


In [31]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2

# Load XML
xml_file = "annotations.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

# Iterate over images
for image in root.findall("image"):
    image_name = image.get("name")
    width = int(image.get("width"))
    height = int(image.get("height"))

    # Create blank mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Iterate over polygons
    for polygon in image.findall("polygon"):
        points_str = polygon.get("points")  # "x1,y1;x2,y2;..."
        points = np.array([[float(xy.split(',')[0]), float(xy.split(',')[1])]
                           for xy in points_str.split(';')], np.int32)
        points = points.reshape((-1, 1, 2))

        # Draw filled polygon
        cv2.fillPoly(mask, [points], color=255)

    # Save mask
    cv2.imwrite(f"{image_name}_mask.png", mask)
    print(f"Mask saved for {image_name}")


Mask saved for 1.png
Mask saved for 10.png
Mask saved for 11.png
Mask saved for 18.png
Mask saved for 2.png
Mask saved for 21.png
Mask saved for 3.png
Mask saved for 4.png
Mask saved for 5.png
Mask saved for 6.png
Mask saved for 7.png
Mask saved for 8.png
Mask saved for 9.png


### **High-Precision Mask Generation (PIL Method)**:

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image, ImageDraw

# Load XML
xml_file = "annotations.xml"
tree = ET.parse(xml_file)
root = tree.getroot()

# Precision scale factor
scale = 8  # Increase for more sub-pixel precision

# Iterate over images
for image in root.findall("image"):
    image_name = image.get("name")
    width = int(image.get("width"))
    height = int(image.get("height"))

    # Create high-resolution blank mask
    hr_size = (width*scale, height*scale)
    mask_hr = Image.new("L", hr_size, 0)  # "L" = grayscale
    draw = ImageDraw.Draw(mask_hr)

    # Draw polygons
    for polygon in image.findall("polygon"):
        points_str = polygon.get("points")  # "x1,y1;x2,y2;..."
        points = [[float(xy.split(',')[0])*scale, float(xy.split(',')[1])*scale]
                  for xy in points_str.split(';')]
        draw.polygon(points, fill=255)

    # Downscale back to original size with anti-aliasing
    mask = mask_hr.resize((width, height), Image.Resampling.LANCZOS)

    # Save mask
    mask.save(f"{image_name}_mask.png")
    print(f"High-precision mask saved for {image_name}")


High-precision mask saved for 1.png
High-precision mask saved for 10.png
High-precision mask saved for 11.png
High-precision mask saved for 12.png
High-precision mask saved for 13.png
High-precision mask saved for 14.png
High-precision mask saved for 15.png
High-precision mask saved for 16.png
High-precision mask saved for 17.png
High-precision mask saved for 18.png
High-precision mask saved for 19.png
High-precision mask saved for 2.png
High-precision mask saved for 20.png
High-precision mask saved for 3.png
High-precision mask saved for 4.png
High-precision mask saved for 5.png
High-precision mask saved for 6.png
High-precision mask saved for 7.png
High-precision mask saved for 8.png
High-precision mask saved for 9.png


**Basic Metallic Texture Application** :

In [ ]:
import cv2
import numpy as np


mask = cv2.imread("10.png_mask.png", cv2.IMREAD_GRAYSCALE)

# Ensure mask is binary
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

#Generate brushed metal texture
h, w = mask.shape
noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)
metal_texture = cv2.GaussianBlur(noise, (51, 1), sigmaX=0)
metal_texture = cv2.cvtColor(metal_texture, cv2.COLOR_GRAY2BGR)

# Apply mask
metallic = cv2.bitwise_and(metal_texture, metal_texture, mask=mask)

# Highlights
highlight = cv2.GaussianBlur(metallic, (0, 0), sigmaX=25, sigmaY=25)
metallic = cv2.addWeighted(metallic, 0.7, highlight, 0.3, 0)

#Contrast boost
metallic = cv2.convertScaleAbs(metallic, alpha=1.4, beta=20)

cv2.imwrite("metallic_output.png", metallic)
print(" Metallic render saved as metallic_output.png")


 Metallic render saved as metallic_output.png


**Enhanced Metallic with Lighting Effects**

In [ ]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("3.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

#Brushed metal base
noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)
metal_texture = cv2.GaussianBlur(noise, (61, 1), sigmaX=0)
metal_texture = cv2.cvtColor(metal_texture, cv2.COLOR_GRAY2BGR)

# Apply mask
metallic = cv2.bitwise_and(metal_texture, metal_texture, mask=mask)

# Add fake lighting (emboss effect on edges)
edges = cv2.Canny(mask, 50, 150)
edges_dilated = cv2.dilate(edges, np.ones((3,3), np.uint8))
highlight = cv2.GaussianBlur(edges_dilated, (15, 15), 5)
highlight = cv2.cvtColor(highlight, cv2.COLOR_GRAY2BGR)

metallic = cv2.addWeighted(metallic, 1.0, highlight, 0.6, 0)

# Add shine (radial gradient)
X, Y = np.meshgrid(np.linspace(-1,1,w), np.linspace(-1,1,h))
radial = np.sqrt(X**2 + Y**2)
radial = (1 - np.clip(radial, 0, 1)) * 255
radial = cv2.GaussianBlur(radial.astype(np.uint8), (101,101), 0)
radial = cv2.cvtColor(radial, cv2.COLOR_GRAY2BGR)

metallic = cv2.addWeighted(metallic, 0.8, radial, 0.4, 0)

#Add color tint (try steel, gold, copper)
tint = np.full_like(metallic, (200, 180, 120))  # gold tone
metallic = cv2.addWeighted(metallic, 0.8, tint, 0.2, 0)

#Contrast & polish
metallic = cv2.convertScaleAbs(metallic, alpha=1.3, beta=15)

cv2.imwrite("metallic_realistic.png", metallic)
print("More realistic metallic render saved as metallic_realistic.png")


More realistic metallic render saved as metallic_realistic.png


 **Cylindrical Surface Geometry**

In [32]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("10.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

# Find the bounding box and center of the object
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    # Get the largest contour (assuming it's our main object)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
    center_x = x + obj_w // 2
    center_y = y + obj_h // 2
else:
    # Fallback to image center
    center_x, center_y = w // 2, h // 2
    obj_w, obj_h = w, h

# Create coordinate grids
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Creating cylindrical coordinate system
# Calculate distance from center line (for cylindrical effect)
dx = X - center_x
dy = Y - center_y

# For a vertical cylinder, use horizontal distance from center
cylinder_radius = np.abs(dx)
max_radius = obj_w // 2

# Normalize to 0-1 range
normalized_radius = np.clip(cylinder_radius / max_radius, 0, 1)

#Create circular brushed metal texture
# Generate noise
noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)

# Create circular brushing by blurring along circular paths

angles = np.arctan2(dy, dx)

# Create multiple texture layers for realism
base_texture = cv2.GaussianBlur(noise, (3, 3), 0)

# Add radial texture lines (like real brushed metal)
radial_lines = np.zeros((h, w), dtype=np.uint8)
for angle in np.linspace(0, 2*np.pi, 120):  # 120 radial lines
    # Create thin radial lines from center
    line_x = center_x + np.cos(angle) * np.arange(0, max_radius + 50)
    line_y = center_y + np.sin(angle) * np.arange(0, max_radius + 50)

    # Ensure coordinates are within bounds
    valid_idx = (line_x >= 0) & (line_x < w) & (line_y >= 0) & (line_y < h)
    line_x = line_x[valid_idx].astype(int)
    line_y = line_y[valid_idx].astype(int)

    radial_lines[line_y, line_x] = 255

# Blur the radial lines to create brushed effect
radial_texture = cv2.GaussianBlur(radial_lines, (5, 5), 0)

# Combine textures
metal_texture = cv2.addWeighted(base_texture, 0.7, radial_texture, 0.3, 0)

# Apply cylindrical lighting (Lambert's law)
# Assume light source from top-left
light_x, light_y, light_z = -0.5, -0.5, 1.0  # Light direction
light_magnitude = np.sqrt(light_x**2 + light_y**2 + light_z**2)
light_x /= light_magnitude
light_y /= light_magnitude
light_z /= light_magnitude

# Calculate surface normals for cylinder
# For a vertical cylinder, normal vectors point radially outward
normal_x = dx / (max_radius + 1e-6)  # Avoid division by zero
normal_y = np.zeros_like(normal_x)
normal_z = np.sqrt(1 - np.clip(normalized_radius**2, 0, 1))

# Calculate dot product (Lambert's cosine law)
dot_product = normal_x * light_x + normal_y * light_y + normal_z * light_z
lighting = np.clip(dot_product, 0.1, 1.0)  # Clamp to avoid pure black

# Apply lighting to texture
lit_texture = (metal_texture * lighting).astype(np.uint8)

#Add specular highlights
# Specular reflection calculation
view_x, view_y, view_z = 0, 0, 1  # Viewer looking straight down

# Reflection vector: R = 2(N·L)N - L
reflect_x = 2 * dot_product * normal_x - light_x
reflect_y = 2 * dot_product * normal_y - light_y
reflect_z = 2 * dot_product * normal_z - light_z

# Specular intensity
specular_dot = np.clip(reflect_x * view_x + reflect_y * view_y + reflect_z * view_z, 0, 1)
specular = np.power(specular_dot, 50) * 255  # High shininess

#Create final metallic surface
# Convert to BGR
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR)

# Add specular highlights
specular_bgr = cv2.cvtColor(specular.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.addWeighted(metallic, 0.8, specular_bgr, 0.4, 0)

#Add metallic color tint
# Steel/aluminum tint
tint = np.full_like(metallic, (200, 195, 185))  # Cool metallic tone
metallic = cv2.addWeighted(metallic, 0.85, tint, 0.15, 0)

#Apply mask and enhance edges
# Apply mask to all channels
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

# Add edge enhancement for realism
edges = cv2.Canny(mask, 50, 150)
edges_dilated = cv2.dilate(edges, np.ones((2, 2), np.uint8))
edge_highlight = cv2.GaussianBlur(edges_dilated, (7, 7), 2)
edge_highlight_bgr = cv2.cvtColor(edge_highlight, cv2.COLOR_GRAY2BGR)

metallic = cv2.addWeighted(metallic, 1.0, edge_highlight_bgr, 0.3, 0)

#Final contrast and polish
metallic = cv2.convertScaleAbs(metallic, alpha=1.2, beta=10)

# Apply mask one final time to ensure clean edges
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

cv2.imwrite("cylindrical_metallic.png", metallic)
print(" Cylindrical metallic render saved as cylindrical_metallic.png")

 Cylindrical metallic render saved as cylindrical_metallic.png


**Advanced Machining Texture**:

In [33]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("10.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

# Create coordinate grids
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Find contours to analyze the shape structure
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
    center_x = x + obj_w // 2
    center_y = y + obj_h // 2
else:
    center_x, center_y = w // 2, h // 2
    obj_w, obj_h = w, h

# === Step 1: Analyze depth/geometry from mask ===
# Use distance transform to estimate depth/radius at each point
dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
max_dist = np.max(dist_transform)

# Normalize to create depth map (0 = edge, 1 = center)
depth_map = dist_transform / (max_dist + 1e-6)

# === Step 2: Create horizontal machining marks (lathe marks) ===
# Generate base noise
noise = np.random.randint(0, 50, (h, w), dtype=np.uint8)

# Create horizontal machining lines - this is key for realistic appearance
machining_lines = np.zeros((h, w), dtype=np.uint8)

# Add multiple sets of horizontal lines at different intensities
for y_pos in range(0, h, 2):  # Every 2 pixels
    machining_lines[y_pos, :] = 30 + np.random.randint(0, 20)

for y_pos in range(0, h, 8):  # Every 8 pixels - stronger lines
    machining_lines[y_pos, :] = 60 + np.random.randint(0, 30)

# Blur slightly to make it look more realistic
machining_lines = cv2.GaussianBlur(machining_lines, (1, 3), 0)

# Combine with noise
base_texture = cv2.addWeighted(noise, 0.6, machining_lines, 0.4, 0)

# === Step 3: Create 3D cylindrical lighting ===
# Calculate distance from center line (horizontal distance for vertical cylinders)
dx = X - center_x
dy = Y - center_y

# For each row, calculate the effective radius based on the mask
effective_radius = np.zeros((h, w))
for row in range(h):
    mask_row = mask[row, :]
    if np.any(mask_row):
        # Find the extent of the mask in this row
        left_edge = np.where(mask_row > 0)[0][0] if np.any(mask_row) else center_x
        right_edge = np.where(mask_row > 0)[0][-1] if np.any(mask_row) else center_x
        row_radius = (right_edge - left_edge) / 2

        # Create cylindrical normal calculation
        for col in range(w):
            if mask[row, col] > 0:
                dist_from_center = abs(col - center_x)
                if row_radius > 0:
                    normalized_dist = dist_from_center / row_radius
                    effective_radius[row, col] = np.clip(normalized_dist, 0, 1)

# Calculate surface normals for cylinder
# Normal points outward from the cylinder axis
normal_x = np.sign(dx) * effective_radius
normal_y = np.zeros_like(normal_x)
normal_z = np.sqrt(1 - np.clip(effective_radius**2, 0, 1))

# Light source position (from upper left, slightly forward)
light_x, light_y, light_z = -0.3, -0.5, 0.8
light_magnitude = np.sqrt(light_x**2 + light_y**2 + light_z**2)
light_x /= light_magnitude
light_y /= light_magnitude
light_z /= light_magnitude

# Lambert lighting calculation
dot_product = normal_x * light_x + normal_y * light_y + normal_z * light_z
# Ensure minimum ambient lighting
lighting = np.clip(dot_product, 0.3, 1.0)

# Apply lighting to texture
lit_texture = (base_texture * lighting).astype(np.uint8)

# === Step 4: Add realistic specular highlights ===
# View vector (looking straight at the object)
view_x, view_y, view_z = 0, 0, 1

# Calculate reflection vector
reflect_x = 2 * dot_product * normal_x - light_x
reflect_y = 2 * dot_product * normal_y - light_y
reflect_z = 2 * dot_product * normal_z - light_z

# Specular component with high shininess for metal
specular_dot = np.clip(reflect_x * view_x + reflect_y * view_y + reflect_z * view_z, 0, 1)
specular = np.power(specular_dot, 80) * 120  # High shininess, moderate intensity

# === Step 5: Add edge enhancement (sharp machined edges) ===
# Detect edges and create sharp transitions
edges = cv2.Canny(mask, 50, 150)
edge_blur = cv2.GaussianBlur(edges, (3, 3), 1)

# Create edge lighting effect
edge_lighting = edge_blur.astype(float) * 0.5

# === Step 6: Combine all elements ===
# Convert to BGR
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR)

# Add specular highlights
specular_bgr = cv2.cvtColor(specular.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.addWeighted(metallic, 0.7, specular_bgr, 0.5, 0)

# Add edge enhancement
edge_bgr = cv2.cvtColor(edge_lighting.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.add(metallic, edge_bgr)

# === Step 7: Apply realistic metal color ===
# Steel/aluminum color (slightly blue-tinted)
tint = np.full_like(metallic, (180, 175, 165))  # Cool steel tone
metallic = cv2.addWeighted(metallic, 0.8, tint, 0.2, 0)

# === Step 8: Apply mask and final processing ===
# Apply mask to all channels
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

# Final contrast adjustment
metallic = cv2.convertScaleAbs(metallic, alpha=1.1, beta=5)

# Ensure clean edges
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

cv2.imwrite("machined_metallic.png", metallic)
print(" Machined metallic part saved as machined_metallic.png")

 Machined metallic part saved as machined_metallic.png


 Ultra-Realistic Surface Modeling

In [34]:
import cv2
import numpy as np
from scipy import ndimage

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')


# Find contours and analyze the part geometry
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
center_x = x + obj_w // 2
center_y = y + obj_h // 2

# Advanced depth estimation using multiple passes
dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
max_dist = np.max(dist_transform)

# Create detailed geometry analysis
geometry_map = np.zeros((h, w), dtype=np.float32)
radius_map = np.zeros((h, w), dtype=np.float32)

# Analyze each horizontal slice to understand the part's profile
for row in range(h):
    mask_row = mask[row, :]
    if np.any(mask_row):
        # Find edges of this slice
        nonzero_cols = np.where(mask_row > 0)[0]
        if len(nonzero_cols) > 0:
            left_edge = nonzero_cols[0]
            right_edge = nonzero_cols[-1]
            slice_width = right_edge - left_edge
            slice_center = (left_edge + right_edge) / 2
            slice_radius = slice_width / 2

            # Create smooth radius variation for this slice
            for col in nonzero_cols:
                dist_from_slice_center = abs(col - slice_center)
                if slice_radius > 0:
                    # Normalized distance from center (0 = center, 1 = edge)
                    norm_dist = dist_from_slice_center / slice_radius
                    radius_map[row, col] = norm_dist

                    # Geometry mapping for different surface types
                    geometry_map[row, col] = slice_radius


# Create base metal grain structure
np.random.seed(42)  # For reproducible results
fine_noise = np.random.normal(128, 15, (h, w)).astype(np.uint8)
fine_noise = cv2.GaussianBlur(fine_noise, (3, 3), 0)

# Horizontal machining marks (lathe turning marks)
machining_texture = np.zeros((h, w), dtype=np.float32)

# Primary machining lines (every 1-2 pixels)
for y_offset in range(0, h, 1):
    intensity = 0.3 + 0.2 * np.sin(y_offset * 0.5)  # Vary intensity
    machining_texture[y_offset, :] += intensity

# Secondary deeper cuts (every 4-6 pixels)
for y_offset in range(0, h, 5):
    intensity = 0.5 + 0.3 * np.random.random()
    if y_offset < h:
        machining_texture[y_offset, :] += intensity
        if y_offset + 1 < h:
            machining_texture[y_offset + 1, :] += intensity * 0.7

# Tertiary tool marks (every 12-15 pixels)
for y_offset in range(0, h, 13):
    intensity = 0.8 + 0.4 * np.random.random()
    if y_offset < h:
        machining_texture[y_offset, :] += intensity

# Blur slightly to simulate tool width
machining_texture = cv2.GaussianBlur(machining_texture, (1, 3), 0)

# Combine base texture with machining marks
base_texture = fine_noise.astype(np.float32) + machining_texture * 40
base_texture = np.clip(base_texture, 0, 255).astype(np.uint8)


# Light setup (realistic studio lighting)
key_light = np.array([-0.4, -0.6, 0.7])  # Main light from upper left
fill_light = np.array([0.3, -0.2, 0.9])  # Fill light from right
rim_light = np.array([0.1, 0.8, 0.6])   # Rim light from behind

# Normalize light vectors
key_light = key_light / np.linalg.norm(key_light)
fill_light = fill_light / np.linalg.norm(fill_light)
rim_light = rim_light / np.linalg.norm(rim_light)

# Calculate surface normals for cylindrical geometry
dx = X - center_x
normal_x = np.zeros((h, w))
normal_y = np.zeros((h, w))
normal_z = np.zeros((h, w))

# For each point, calculate the normal based on cylindrical surface
for row in range(h):
    for col in range(w):
        if mask[row, col] > 0:
            # Distance from center line
            dist_from_center = col - center_x
            current_radius = geometry_map[row, col]

            if current_radius > 0:
                # Cylindrical normal (pointing radially outward)
                norm_dist = abs(dist_from_center) / current_radius
                norm_dist = np.clip(norm_dist, 0, 1)

                # Calculate normal vector
                normal_x[row, col] = np.sign(dist_from_center) * norm_dist
                normal_y[row, col] = 0  # No Y component for vertical cylinder
                normal_z[row, col] = np.sqrt(1 - norm_dist**2)

# Calculate lighting contributions
key_lighting = np.maximum(0, normal_x * key_light[0] + normal_y * key_light[1] + normal_z * key_light[2])
fill_lighting = np.maximum(0, normal_x * fill_light[0] + normal_y * fill_light[1] + normal_z * fill_light[2])
rim_lighting = np.maximum(0, normal_x * rim_light[0] + normal_y * rim_light[1] + normal_z * rim_light[2])

# Combine lighting (weighted)
total_lighting = (key_lighting * 0.7 + fill_lighting * 0.2 + rim_lighting * 0.3)
total_lighting = np.clip(total_lighting + 0.2, 0.2, 1.0)  # Add ambient

# Apply lighting to base texture
lit_texture = (base_texture.astype(np.float32) * total_lighting).astype(np.uint8)

# === Step 4: Advanced Specular Reflections ===
view_vector = np.array([0, 0, 1])  # Looking straight at object

# Calculate specular for each light source
def calculate_specular(light_vec, normal_x, normal_y, normal_z, shininess):
    # Reflection vector: R = 2(N·L)N - L
    dot_NL = normal_x * light_vec[0] + normal_y * light_vec[1] + normal_z * light_vec[2]
    reflect_x = 2 * dot_NL * normal_x - light_vec[0]
    reflect_y = 2 * dot_NL * normal_y - light_vec[1]
    reflect_z = 2 * dot_NL * normal_z - light_vec[2]

    # Specular intensity
    spec_dot = np.maximum(0, reflect_x * view_vector[0] + reflect_y * view_vector[1] + reflect_z * view_vector[2])
    return np.power(spec_dot, shininess)

# Multiple specular highlights with different characteristics
key_specular = calculate_specular(key_light, normal_x, normal_y, normal_z, 120) * 180
fill_specular = calculate_specular(fill_light, normal_x, normal_y, normal_z, 80) * 100
rim_specular = calculate_specular(rim_light, normal_x, normal_y, normal_z, 200) * 120

total_specular = key_specular + fill_specular * 0.5 + rim_specular * 0.7
total_specular = np.clip(total_specular, 0, 255)


# Edge detection for sharp machined edges
edges = cv2.Canny(mask, 30, 100)
edges_dilated = cv2.dilate(edges, np.ones((2, 2), np.uint8))

# Create edge highlighting
edge_highlight = cv2.GaussianBlur(edges_dilated.astype(np.float32), (5, 5), 1) * 0.8

# Surface scratches and imperfections
scratches = np.zeros((h, w), dtype=np.float32)
for _ in range(15):  # Random scratches
    start_y = np.random.randint(0, h)
    start_x = np.random.randint(0, w)
    end_y = start_y + np.random.randint(-20, 20)
    end_x = start_x + np.random.randint(-50, 50)

    if 0 <= end_y < h and 0 <= end_x < w:
        cv2.line(scratches, (start_x, start_y), (end_x, end_y), 0.3, 1)

scratches = cv2.GaussianBlur(scratches, (3, 3), 0) # Changed (2, 2) to (3, 3)


# Convert to color
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR).astype(np.float32)

# Add specular highlights
for i in range(3):
    metallic[:, :, i] += total_specular

# Add edge enhancement
for i in range(3):
    metallic[:, :, i] += edge_highlight

# Subtract scratches (darker lines)
for i in range(3):
    metallic[:, :, i] -= scratches * 30

# Realistic steel color grading
steel_tint = np.array([195, 190, 180])  # Slightly warm steel
for i in range(3):
    metallic[:, :, i] = metallic[:, :, i] * 0.85 + steel_tint[i] * 0.15

# Clamp values
metallic = np.clip(metallic, 0, 255).astype(np.uint8)

# Apply mask
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

# Final contrast and sharpening
metallic = cv2.convertScaleAbs(metallic, alpha=1.05, beta=3)

# Subtle sharpening for machined surface detail
kernel = np.array([[-0.1, -0.1, -0.1],
                   [-0.1,  1.8, -0.1],
                   [-0.1, -0.1, -0.1]])
for i in range(3):
    metallic[:, :, i] = cv2.filter2D(metallic[:, :, i], -1, kernel)
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

metallic = np.clip(metallic, 0, 255).astype(np.uint8)

cv2.imwrite("ultra_realistic_metallic.png", metallic)
print(" Ultra-realistic machined metallic part saved as ultra_realistic_metallic.png")

 Ultra-realistic machined metallic part saved as ultra_realistic_metallic.png


In [ ]:
!pip install perlin-noise

 Culmination of all previous experiments, incorporating the most successful
 techniques into a production-ready function with proper error handling and parameter control.

In [ ]:
import cv2
import numpy as np
import os

#  Try importing Perlin noise, fallback if not installed
try:
    from perlin_noise import PerlinNoise
    noise_available = True
except ImportError:
    print("[INFO] PerlinNoise not found. Falling back to Gaussian noise.")
    noise_available = False


def generate_metallic(mask_path, output_path="output_metal.png"):
    # Load binary mask (white = object, black = background)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(f"Could not load mask: {mask_path}")

    h, w = mask.shape
    metallic = np.zeros((h, w, 3), dtype=np.uint8)

    # Base metal color
    base_color = np.array([192, 192, 192])  # silver/steel look
    metallic[mask > 0] = base_color

    # Add surface texture
    if noise_available:
        noise = PerlinNoise(octaves=6)
        for y in range(h):
            for x in range(w):
                if mask[y, x] > 0:
                    n_val = noise([x / w, y / h])
                    shade = int((n_val + 1) * 127)  # scale to [0..255]
                    metallic[y, x] = base_color - [shade//6, shade//6, shade//6]
    else:
        # Gaussian noise fallback
        noise = np.random.normal(0, 15, (h, w, 3)).astype(np.int16)
        metallic = np.clip(metallic + noise, 0, 255).astype(np.uint8)

    #  Add highlights & reflections
    blur = cv2.GaussianBlur(metallic, (0, 0), sigmaX=3)
    metallic = cv2.addWeighted(metallic, 0.8, blur, 0.2, 0)

    # Save output
    cv2.imwrite(output_path, metallic)
    print(f"[OK] Metallic image saved as {output_path}")


#  Run the function
if __name__ == "__main__":
    # Replace with your binary mask path
    generate_metallic("6.png_mask.png")


[OK] Metallic image saved as output_metal.png


In [ ]:
import cv2
import numpy as np
from scipy import ndimage
import math

def generate_photorealistic_metallic(mask_path, output_path="photorealistic_metal.png"):
    """
    Generate photorealistic metallic rendering based on reference image analysis.
    Focuses on graduated cylindrical lighting, fine surface textures, and authentic material appearance.
    """

    # Load and prepare mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise FileNotFoundError(f"Could not load mask: {mask_path}")

    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    h, w = mask.shape

    # Create coordinate grids
    Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

    # Advanced geometric analysis
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
        center_x = x + obj_w // 2
        center_y = y + obj_h // 2
    else:
        center_x, center_y = w // 2, h // 2
        obj_w, obj_h = w, h

    print(f"Processing object: {obj_w}x{obj_h} pixels, center at ({center_x}, {center_y})")


    # Create sophisticated radius mapping for each row
    radius_map = np.zeros((h, w), dtype=np.float32)
    depth_map = np.zeros((h, w), dtype=np.float32)

    for row in range(h):
        mask_row = mask[row, :]
        if np.any(mask_row):
            # Find precise edges for this row
            nonzero_indices = np.where(mask_row > 0)[0]
            if len(nonzero_indices) > 1:
                left_edge = nonzero_indices[0]
                right_edge = nonzero_indices[-1]
                row_width = right_edge - left_edge
                row_center = (left_edge + right_edge) / 2
                row_radius = row_width / 2

                # Create smooth cylindrical mapping
                for col in nonzero_indices:
                    dist_from_center = abs(col - row_center)
                    if row_radius > 0:
                        # Normalized distance from center (0=center, 1=edge)
                        norm_dist = np.clip(dist_from_center / row_radius, 0, 1)
                        radius_map[row, col] = norm_dist
                        # Depth based on cylindrical geometry
                        depth_map[row, col] = np.sqrt(1 - norm_dist**2)

    #  PHOTOREALISTIC SURFACE TEXTURE
    # Base metal grain structure (very fine)
    np.random.seed(42)  # Reproducible results
    base_grain = np.random.normal(128, 8, (h, w)).astype(np.float32)
    base_grain = cv2.GaussianBlur(base_grain, (3, 3), 0.5)

    # Fine horizontal machining marks (key for realism)
    machining_texture = np.zeros((h, w), dtype=np.float32)

    # Primary lathe marks - very fine, every 1-2 pixels
    for y in range(0, h, 1):
        # Vary intensity slightly for realism
        intensity = 0.15 + 0.05 * np.sin(y * 0.3) + 0.03 * np.random.random()
        if y < h:
            machining_texture[y, :] += intensity

    # Secondary marks slightly deeper, every 3-4 pixels
    for y in range(0, h, 3):
        intensity = 0.25 + 0.1 * np.random.random()
        if y < h:
            machining_texture[y, :] += intensity
            # Slight spread for tool width simulation
            if y + 1 < h:
                machining_texture[y + 1, :] += intensity * 0.6

    # Occasional deeper tool marks
    for y in range(0, h, 12):
        if np.random.random() > 0.7:  # Random occurrence
            intensity = 0.4 + 0.2 * np.random.random()
            if y < h:
                machining_texture[y, :] += intensity

    # Apply subtle blur to simulate finite tool width
    machining_texture = cv2.GaussianBlur(machining_texture, (1, 3), 0)

    # Combine textures
    surface_texture = base_grain + machining_texture * 25
    surface_texture = np.clip(surface_texture, 0, 255)

    #  SOPHISTICATED LIGHTING MODEL
    # Calculate precise surface normals
    dx = X - center_x
    normal_x = np.zeros((h, w), dtype=np.float32)
    normal_y = np.zeros((h, w), dtype=np.float32)
    normal_z = np.zeros((h, w), dtype=np.float32)

    # For cylindrical surfaces
    mask_bool = mask > 0
    normal_x[mask_bool] = np.sign(dx[mask_bool]) * radius_map[mask_bool]
    normal_y[mask_bool] = 0.0  # Vertical cylinder
    normal_z[mask_bool] = depth_map[mask_bool]

    # Professional lighting setup (matching reference image)
    # Main key light from upper left
    key_light = np.array([-0.5, -0.7, 0.5])
    key_light = key_light / np.linalg.norm(key_light)

    # Soft fill light from right
    fill_light = np.array([0.4, -0.3, 0.85])
    fill_light = fill_light / np.linalg.norm(fill_light)

    # Calculate lighting contributions
    key_dot = np.maximum(0, normal_x * key_light[0] + normal_y * key_light[1] + normal_z * key_light[2])
    fill_dot = np.maximum(0, normal_x * fill_light[0] + normal_y * fill_light[1] + normal_z * fill_light[2])

    # Combine with proper weighting
    diffuse_lighting = key_dot * 0.8 + fill_dot * 0.3
    # Add ambient component
    diffuse_lighting = np.clip(diffuse_lighting + 0.25, 0.25, 1.0)

    # Apply lighting to surface texture
    lit_surface = (surface_texture * diffuse_lighting).astype(np.uint8)

    #  REALISTIC SPECULAR REFLECTIONS
    view_dir = np.array([0, 0, 1])  # Viewer looking straight at surface

    # Calculate specular for key light (main highlight)
    # Reflection vector: R = 2(N·L)N - L
    reflect_x = 2 * key_dot * normal_x - key_light[0]
    reflect_y = 2 * key_dot * normal_y - key_light[1]
    reflect_z = 2 * key_dot * normal_z - key_light[2]

    # Specular intensity
    spec_dot = np.maximum(0, reflect_x * view_dir[0] + reflect_y * view_dir[1] + reflect_z * view_dir[2])
    specular = np.power(spec_dot, 60) * 140  # Moderate shininess for machined metal

    # Softer specular from fill light
    fill_reflect_x = 2 * fill_dot * normal_x - fill_light[0]
    fill_reflect_y = 2 * fill_dot * normal_y - fill_light[1]
    fill_reflect_z = 2 * fill_dot * normal_z - fill_light[2]

    fill_spec_dot = np.maximum(0, fill_reflect_x * view_dir[0] + fill_reflect_y * view_dir[1] + fill_reflect_z * view_dir[2])
    fill_specular = np.power(fill_spec_dot, 40) * 80

    total_specular = specular + fill_specular * 0.6

    #  EDGE ENHANCEMENT (SUBTLE BUT IMPORTANT)
    # Detect edges with multiple scales
    edges_fine = cv2.Canny(mask, 50, 100)
    edges_coarse = cv2.Canny(mask, 30, 80)

    # Create soft edge enhancement
    edge_enhance = cv2.GaussianBlur(edges_fine.astype(np.float32), (3, 3), 1) * 0.3
    edge_enhance += cv2.GaussianBlur(edges_coarse.astype(np.float32), (5, 5), 2) * 0.2

    #  AUTHENTIC MATERIAL COLOR ===
    # Convert to BGR for color processing
    metallic = cv2.cvtColor(lit_surface, cv2.COLOR_GRAY2BGR).astype(np.float32)

    # Add specular highlights
    for i in range(3):
        metallic[:, :, i] += total_specular

    # Add subtle edge enhancement
    for i in range(3):
        metallic[:, :, i] += edge_enhance

    # Authentic steel/aluminum color grading (matching reference)
    # Steel has a very subtle blue-gray tint
    steel_color = np.array([185, 188, 192])  # BGR: slightly blue-tinted steel

    # Apply color grading
    for i in range(3):
        metallic[:, :, i] = metallic[:, :, i] * 0.82 + steel_color[i] * 0.18

    # SURFACE IMPERFECTIONS (VERY SUBTLE)
    # Add minimal surface variations for ultra-realism
    surface_noise = np.random.normal(0, 2, (h, w, 3)).astype(np.float32)
    surface_noise = cv2.GaussianBlur(surface_noise, (5, 5), 1)
    metallic += surface_noise

    # Clamp values and apply mask
    metallic = np.clip(metallic, 0, 255).astype(np.uint8)

    # Apply mask to all channels
    for i in range(3):
        metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

    #  FINAL PROFESSIONAL POLISH
    # Subtle contrast adjustment (avoid over-processing)
    metallic = cv2.convertScaleAbs(metallic, alpha=1.08, beta=2)

    # Very gentle sharpening to enhance fine details
    sharpen_kernel = np.array([[-0.05, -0.1, -0.05],
                              [-0.1,  1.5, -0.1],
                              [-0.05, -0.1, -0.05]])

    for i in range(3):
        channel = cv2.filter2D(metallic[:, :, i], -1, sharpen_kernel)
        metallic[:, :, i] = cv2.bitwise_and(channel, channel, mask=mask)

    # Final clamp and mask application
    metallic = np.clip(metallic, 0, 255).astype(np.uint8)

    for i in range(3):
        metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)


    cv2.imwrite(output_path, metallic)
    print(f" Photorealistic metallic rendering saved as {output_path}")

    return metallic


if __name__ == "__main__":
    # Process with the reference image characteristics in mind
    result = generate_photorealistic_metallic("10.png_mask.png", "photorealistic_metallic_v2.png")

    print("\\n=== Rendering Complete ===")
    print("Key improvements based on reference image:")
    print(" Graduated cylindrical lighting with smooth falloff")
    print(" Fine horizontal machining marks (lathe texture)")
    print(" Authentic steel color grading")
    print(" Professional lighting setup with key and fill lights")
    print("Realistic specular reflections with appropriate shininess")
    print(" Subtle surface imperfections for ultra-realism")
    print(" Sharp but natural edge definition")

Processing object: 3325x523 pixels, center at (1723, 947)
 Photorealistic metallic rendering saved as photorealistic_metallic_v2.png
\n=== Rendering Complete ===
Key improvements based on reference image:
 Graduated cylindrical lighting with smooth falloff
 Fine horizontal machining marks (lathe texture)
 Authentic steel color grading
 Professional lighting setup with key and fill lights
Realistic specular reflections with appropriate shininess
 Subtle surface imperfections for ultra-realism
 Sharp but natural edge definition


In [ ]:
import cv2
import numpy as np
from scipy import ndimage

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Find contours and analyze the part geometry
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
center_x = x + obj_w // 2
center_y = y + obj_h // 2

# Advanced depth estimation using multiple passes
dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
max_dist = np.max(dist_transform)

# Create detailed geometry analysis
geometry_map = np.zeros((h, w), dtype=np.float32)
radius_map = np.zeros((h, w), dtype=np.float32)

# Analyze each horizontal slice to understand the part's profile
for row in range(h):
    mask_row = mask[row, :]
    if np.any(mask_row):
        nonzero_cols = np.where(mask_row > 0)[0]
        if len(nonzero_cols) > 0:
            left_edge = nonzero_cols[0]
            right_edge = nonzero_cols[-1]
            slice_width = right_edge - left_edge
            slice_center = (left_edge + right_edge) / 2
            slice_radius = slice_width / 2

            for col in nonzero_cols:
                dist_from_slice_center = abs(col - slice_center)
                if slice_radius > 0:
                    norm_dist = dist_from_slice_center / slice_radius
                    radius_map[row, col] = norm_dist
                    geometry_map[row, col] = slice_radius

# Create base metal grain structure (finer for high-quality machining)
np.random.seed(42)
fine_noise = np.random.normal(140, 8, (h, w)).astype(np.uint8)  # Brighter base
fine_noise = cv2.GaussianBlur(fine_noise, (3, 3), 0)

# Horizontal machining marks (very fine, like precision turning)
machining_texture = np.zeros((h, w), dtype=np.float32)

# Very fine machining lines (every pixel for smooth finish)
for y_offset in range(0, h, 1):
    intensity = 0.15 + 0.1 * np.sin(y_offset * 0.3)
    machining_texture[y_offset, :] += intensity

# Occasional deeper tool marks
for y_offset in range(0, h, 8):
    intensity = 0.2 + 0.1 * np.random.random()
    if y_offset < h:
        machining_texture[y_offset, :] += intensity

# Very subtle blur for realistic tool finish
machining_texture = cv2.GaussianBlur(machining_texture, (1, 3), 0)

# Combine base texture with much subtler machining marks
base_texture = fine_noise.astype(np.float32) + machining_texture * 20
base_texture = np.clip(base_texture, 0, 255).astype(np.uint8)

# Professional studio lighting setup (brighter, even, minimal shadows)
key_light = np.array([-0.2, -0.3, 0.9])    # Main light from front-left, mostly frontal
fill_light = np.array([0.2, -0.1, 0.95])   # Fill light from front-right
ambient_strength = 0.6  # Strong ambient lighting

# Normalize light vectors
key_light = key_light / np.linalg.norm(key_light)
fill_light = fill_light / np.linalg.norm(fill_light)

# Calculate surface normals for cylindrical geometry
normal_x = np.zeros((h, w))
normal_y = np.zeros((h, w))
normal_z = np.zeros((h, w))

for row in range(h):
    for col in range(w):
        if mask[row, col] > 0:
            dist_from_center = col - center_x
            current_radius = geometry_map[row, col]

            if current_radius > 0:
                norm_dist = abs(dist_from_center) / current_radius
                norm_dist = np.clip(norm_dist, 0, 1)

                normal_x[row, col] = np.sign(dist_from_center) * norm_dist
                normal_y[row, col] = 0
                normal_z[row, col] = np.sqrt(1 - norm_dist**2)

# Calculate lighting contributions (brighter, more even)
key_lighting = np.maximum(0, normal_x * key_light[0] + normal_y * key_light[1] + normal_z * key_light[2])
fill_lighting = np.maximum(0, normal_x * fill_light[0] + normal_y * fill_light[1] + normal_z * fill_light[2])

# Combine lighting with strong ambient
total_lighting = (key_lighting * 0.4 + fill_lighting * 0.4 + ambient_strength)
total_lighting = np.clip(total_lighting, 0.4, 1.2)

# Apply lighting to base texture
lit_texture = (base_texture.astype(np.float32) * total_lighting).astype(np.uint8)

# Professional specular highlights (brighter, sharp)
view_vector = np.array([0, 0, 1])

def calculate_specular(light_vec, normal_x, normal_y, normal_z, shininess):
    dot_NL = normal_x * light_vec[0] + normal_y * light_vec[1] + normal_z * light_vec[2]
    reflect_x = 2 * dot_NL * normal_x - light_vec[0]
    reflect_y = 2 * dot_NL * normal_y - light_vec[1]
    reflect_z = 2 * dot_NL * normal_z - light_vec[2]

    spec_dot = np.maximum(0, reflect_x * view_vector[0] + reflect_y * view_vector[1] + reflect_z * view_vector[2])
    return np.power(spec_dot, shininess)

# Bright, professional specular highlights
key_specular = calculate_specular(key_light, normal_x, normal_y, normal_z, 150) * 200
fill_specular = calculate_specular(fill_light, normal_x, normal_y, normal_z, 120) * 150

total_specular = key_specular + fill_specular * 0.6
total_specular = np.clip(total_specular, 0, 255)

# Edge detection for precision edges
edges = cv2.Canny(mask, 50, 150)
edges_dilated = cv2.dilate(edges, np.ones((1, 1), np.uint8))
edge_highlight = cv2.GaussianBlur(edges_dilated.astype(np.float32), (3, 3), 0.8) * 0.5

# Minimal surface imperfections (high-quality part)
scratches = np.zeros((h, w), dtype=np.float32)
for _ in range(8):  # Fewer scratches for clean part
    start_y = np.random.randint(0, h)
    start_x = np.random.randint(0, w)
    end_y = start_y + np.random.randint(-15, 15)
    end_x = start_x + np.random.randint(-30, 30)

    if 0 <= end_y < h and 0 <= end_x < w:
        cv2.line(scratches, (start_x, start_y), (end_x, end_y), 0.2, 1)

scratches = cv2.GaussianBlur(scratches, (3, 3), 0)

# Convert to color and build the part
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR).astype(np.float32)

# Add bright specular highlights
for i in range(3):
    metallic[:, :, i] += total_specular

# Add subtle edge enhancement
for i in range(3):
    metallic[:, :, i] += edge_highlight

# Very subtle scratches
for i in range(3):
    metallic[:, :, i] -= scratches * 15

# High-quality steel color (bright, clean steel)
steel_tint = np.array([210, 208, 205])  # Bright steel color
for i in range(3):
    metallic[:, :, i] = metallic[:, :, i] * 0.8 + steel_tint[i] * 0.2

metallic = np.clip(metallic, 0, 255).astype(np.uint8)

# Apply mask to part
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)



# Create clean, bright industrial background like the reference
background = np.ones((h, w, 3), dtype=np.uint8) * 240  # Very bright base

# Add vertical panels/structure (like the white panels in reference)
panel_width = w // 4
for i in range(4):
    panel_start = i * panel_width
    panel_end = min(panel_start + panel_width - 10, w)

    # Alternate panel brightness
    if i % 2 == 0:
        panel_color = np.array([245, 245, 245])  # Brighter panels
    else:
        panel_color = np.array([235, 235, 235])  # Slightly darker panels

    background[:, panel_start:panel_end] = panel_color

    # Add panel separators (dark lines like in reference)
    if panel_end < w:
        separator_color = np.array([180, 180, 180])
        background[:, panel_end:panel_end+3] = separator_color

# Add horizontal elements (like equipment/machinery silhouettes)
# Top area - lighter (like the white area in reference)
background[:int(h*0.3), :] = [250, 250, 250]

# Add some subtle geometric elements (machinery outlines)
# Left side equipment outline
equipment_color = np.array([200, 200, 200])
background[int(h*0.1):int(h*0.6), int(w*0.05):int(w*0.15)] = equipment_color

# Right side equipment
background[int(h*0.15):int(h*0.5), int(w*0.85):int(w*0.95)] = equipment_color

# Add subtle lighting gradient (professional studio lighting)
lighting_gradient = np.linspace(1.05, 0.95, h).reshape(-1, 1)
for i in range(3):
    background[:, :, i] = (background[:, :, i] * lighting_gradient).astype(np.uint8)

# Add very subtle shadow from the part
shadow_offset_x, shadow_offset_y = 3, 6
shadow_mask = np.zeros((h, w), dtype=np.float32)

if shadow_offset_y < h and shadow_offset_x < w:
    shadow_mask[shadow_offset_y:, shadow_offset_x:] = mask[:-shadow_offset_y, :-shadow_offset_x]

shadow_mask = cv2.GaussianBlur(shadow_mask, (8, 8), 3) / 255.0
shadow_strength = 0.15  # Very subtle shadow

for i in range(3):
    background[:, :, i] = (background[:, :, i] * (1 - shadow_mask * shadow_strength)).astype(np.uint8)

# Combine part with background
final_image = background.copy()
inverse_mask = cv2.bitwise_not(mask)

# Apply background where part isn't
for i in range(3):
    final_image[:, :, i] = cv2.bitwise_and(background[:, :, i], background[:, :, i], mask=inverse_mask)

# Add the metallic part
for i in range(3):
    part_channel = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)
    final_image[:, :, i] = cv2.add(final_image[:, :, i], part_channel)

# Final professional color grading
final_image = cv2.convertScaleAbs(final_image, alpha=1.08, beta=5)  # Slight brightness boost

# Very subtle contrast enhancement
final_image = cv2.convertScaleAbs(final_image, alpha=1.02, beta=0)

cv2.imwrite("professional_studio_metallic.png", final_image)
print("Professional studio metallic part saved as professional_studio_metallic.png")

error: OpenCV(4.12.0) /io/opencv/modules/imgproc/src/smooth.dispatch.cpp:293: error: (-215:Assertion failed) ksize.width > 0 && ksize.width % 2 == 1 && ksize.height > 0 && ksize.height % 2 == 1 in function 'createGaussianKernels'


In [ ]:
import cv2
import numpy as np
from scipy import ndimage

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Find contours and analyze the part geometry
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
center_x = x + obj_w // 2
center_y = y + obj_h // 2

# Advanced depth estimation using multiple passes
dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
max_dist = np.max(dist_transform)

# Create detailed geometry analysis
geometry_map = np.zeros((h, w), dtype=np.float32)
radius_map = np.zeros((h, w), dtype=np.float32)

# Analyze each horizontal slice to understand the part's profile
for row in range(h):
    mask_row = mask[row, :]
    if np.any(mask_row):
        nonzero_cols = np.where(mask_row > 0)[0]
        if len(nonzero_cols) > 0:
            left_edge = nonzero_cols[0]
            right_edge = nonzero_cols[-1]
            slice_width = right_edge - left_edge
            slice_center = (left_edge + right_edge) / 2
            slice_radius = slice_width / 2

            for col in nonzero_cols:
                dist_from_slice_center = abs(col - slice_center)
                if slice_radius > 0:
                    norm_dist = dist_from_slice_center / slice_radius
                    radius_map[row, col] = norm_dist
                    geometry_map[row, col] = slice_radius

# Create base metal grain structure (finer for high-quality machining)
np.random.seed(42)
fine_noise = np.random.normal(140, 8, (h, w)).astype(np.uint8)  # Brighter base
fine_noise = cv2.GaussianBlur(fine_noise, (3, 3), 0)

# Horizontal machining marks (very fine, like precision turning)
machining_texture = np.zeros((h, w), dtype=np.float32)

# Very fine machining lines (every pixel for smooth finish)
for y_offset in range(0, h, 1):
    intensity = 0.15 + 0.1 * np.sin(y_offset * 0.3)
    machining_texture[y_offset, :] += intensity

# Occasional deeper tool marks
for y_offset in range(0, h, 8):
    intensity = 0.2 + 0.1 * np.random.random()
    if y_offset < h:
        machining_texture[y_offset, :] += intensity

# Very subtle blur for realistic tool finish
machining_texture = cv2.GaussianBlur(machining_texture, (1, 3), 0)

# Combine base texture with much subtler machining marks
base_texture = fine_noise.astype(np.float32) + machining_texture * 20
base_texture = np.clip(base_texture, 0, 255).astype(np.uint8)

# Professional studio lighting setup (brighter, even, minimal shadows)
key_light = np.array([-0.2, -0.3, 0.9])    # Main light from front-left, mostly frontal
fill_light = np.array([0.2, -0.1, 0.95])   # Fill light from front-right
ambient_strength = 0.6  # Strong ambient lighting

# Normalize light vectors
key_light = key_light / np.linalg.norm(key_light)
fill_light = fill_light / np.linalg.norm(fill_light)

# Calculate surface normals for cylindrical geometry
normal_x = np.zeros((h, w))
normal_y = np.zeros((h, w))
normal_z = np.zeros((h, w))

for row in range(h):
    for col in range(w):
        if mask[row, col] > 0:
            dist_from_center = col - center_x
            current_radius = geometry_map[row, col]

            if current_radius > 0:
                norm_dist = abs(dist_from_center) / current_radius
                norm_dist = np.clip(norm_dist, 0, 1)

                normal_x[row, col] = np.sign(dist_from_center) * norm_dist
                normal_y[row, col] = 0
                normal_z[row, col] = np.sqrt(1 - norm_dist**2)

# Calculate lighting contributions (brighter, more even)
key_lighting = np.maximum(0, normal_x * key_light[0] + normal_y * key_light[1] + normal_z * key_light[2])
fill_lighting = np.maximum(0, normal_x * fill_light[0] + normal_y * fill_light[1] + normal_z * fill_light[2])

# Combine lighting with strong ambient
total_lighting = (key_lighting * 0.4 + fill_lighting * 0.4 + ambient_strength)
total_lighting = np.clip(total_lighting, 0.4, 1.2)

# Apply lighting to base texture
lit_texture = (base_texture.astype(np.float32) * total_lighting).astype(np.uint8)

# Professional specular highlights (brighter, sharp)
view_vector = np.array([0, 0, 1])

def calculate_specular(light_vec, normal_x, normal_y, normal_z, shininess):
    dot_NL = normal_x * light_vec[0] + normal_y * light_vec[1] + normal_z * light_vec[2]
    reflect_x = 2 * dot_NL * normal_x - light_vec[0]
    reflect_y = 2 * dot_NL * normal_y - light_vec[1]
    reflect_z = 2 * dot_NL * normal_z - light_vec[2]

    spec_dot = np.maximum(0, reflect_x * view_vector[0] + reflect_y * view_vector[1] + reflect_z * view_vector[2])
    return np.power(spec_dot, shininess)

# Bright, professional specular highlights
key_specular = calculate_specular(key_light, normal_x, normal_y, normal_z, 150) * 200
fill_specular = calculate_specular(fill_light, normal_x, normal_y, normal_z, 120) * 150

total_specular = key_specular + fill_specular * 0.6
total_specular = np.clip(total_specular, 0, 255)

# Edge detection for precision edges
edges = cv2.Canny(mask, 50, 150)
edges_dilated = cv2.dilate(edges, np.ones((1, 1), np.uint8))
edge_highlight = cv2.GaussianBlur(edges_dilated.astype(np.float32), (3, 3), 0.8) * 0.5

# Minimal surface imperfections (high-quality part)
scratches = np.zeros((h, w), dtype=np.float32)
for _ in range(8):  # Fewer scratches for clean part
    start_y = np.random.randint(0, h)
    start_x = np.random.randint(0, w)
    end_y = start_y + np.random.randint(-15, 15)
    end_x = start_x + np.random.randint(-30, 30)

    if 0 <= end_y < h and 0 <= end_x < w:
        cv2.line(scratches, (start_x, start_y), (end_x, end_y), 0.2, 1)

scratches = cv2.GaussianBlur(scratches, (3, 3), 0)

# Convert to color and build the part
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR).astype(np.float32)

# Add bright specular highlights
for i in range(3):
    metallic[:, :, i] += total_specular

# Add subtle edge enhancement
for i in range(3):
    metallic[:, :, i] += edge_highlight

# Very subtle scratches
for i in range(3):
    metallic[:, :, i] -= scratches * 15

# High-quality steel color (bright, clean steel)
steel_tint = np.array([210, 208, 205])  # Bright steel color
for i in range(3):
    metallic[:, :, i] = metallic[:, :, i] * 0.8 + steel_tint[i] * 0.2

metallic = np.clip(metallic, 0, 255).astype(np.uint8)

# Apply mask to part
for i in range(3):
    metallic[:, :, i] = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)

# ===== PROFESSIONAL STUDIO BACKGROUND =====

# Create clean, bright industrial background like the reference
background = np.ones((h, w, 3), dtype=np.uint8) * 240  # Very bright base

# Add vertical panels/structure (like the white panels in reference)
panel_width = w // 4
for i in range(4):
    panel_start = i * panel_width
    panel_end = min(panel_start + panel_width - 10, w)

    # Alternate panel brightness
    if i % 2 == 0:
        panel_color = np.array([245, 245, 245])  # Brighter panels
    else:
        panel_color = np.array([235, 235, 235])  # Slightly darker panels

    background[:, panel_start:panel_end] = panel_color

    # Add panel separators (dark lines like in reference)
    if panel_end < w:
        separator_color = np.array([180, 180, 180])
        background[:, panel_end:panel_end+3] = separator_color

# Add horizontal elements (like equipment/machinery silhouettes)
# Top area - lighter (like the white area in reference)
background[:int(h*0.3), :] = [250, 250, 250]

# Add some subtle geometric elements (machinery outlines)
# Left side equipment outline
equipment_color = np.array([200, 200, 200])
background[int(h*0.1):int(h*0.6), int(w*0.05):int(w*0.15)] = equipment_color

# Right side equipment
background[int(h*0.15):int(h*0.5), int(w*0.85):int(w*0.95)] = equipment_color

# Add subtle lighting gradient (professional studio lighting)
lighting_gradient = np.linspace(1.05, 0.95, h).reshape(-1, 1)
for i in range(3):
    background[:, :, i] = (background[:, :, i] * lighting_gradient).astype(np.uint8)

# Add very subtle shadow from the part
shadow_offset_x, shadow_offset_y = 3, 6
shadow_mask = np.zeros((h, w), dtype=np.float32)

if shadow_offset_y < h and shadow_offset_x < w:
    shadow_mask[shadow_offset_y:, shadow_offset_x:] = mask[:-shadow_offset_y, :-shadow_offset_x]

shadow_mask = cv2.GaussianBlur(shadow_mask, (9, 9), 3) / 255.0
shadow_strength = 0.15  # Very subtle shadow

for i in range(3):
    background[:, :, i] = (background[:, :, i] * (1 - shadow_mask * shadow_strength)).astype(np.uint8)

# Combine part with background
final_image = background.copy()
inverse_mask = cv2.bitwise_not(mask)

# Apply background where part isn't
for i in range(3):
    final_image[:, :, i] = cv2.bitwise_and(background[:, :, i], background[:, :, i], mask=inverse_mask)

# Add the metallic part
for i in range(3):
    part_channel = cv2.bitwise_and(metallic[:, :, i], metallic[:, :, i], mask=mask)
    final_image[:, :, i] = cv2.add(final_image[:, :, i], part_channel)

# Final professional color grading
final_image = cv2.convertScaleAbs(final_image, alpha=1.08, beta=5)  # Slight brightness boost

# Very subtle contrast enhancement
final_image = cv2.convertScaleAbs(final_image, alpha=1.02, beta=0)

cv2.imwrite("professional_studio_metallic.png", final_image)
print("Professional studio metallic part saved as professional_studio_metallic.png")

Professional studio metallic part saved as professional_studio_metallic.png


In [ ]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

# Find the bounding box and center of the object
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    # Get the largest contour (assuming it's our main object)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
    center_x = x + obj_w // 2
    center_y = y + obj_h // 2
else:
    # Fallback to image center
    center_x, center_y = w // 2, h // 2
    obj_w, obj_h = w, h

# Create coordinate grids
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Creating cylindrical coordinate system
# Calculate distance from center line (for cylindrical effect)
dx = X - center_x
dy = Y - center_y

# For a vertical cylinder, use horizontal distance from center
cylinder_radius = np.abs(dx)
max_radius = obj_w // 2

# Normalize to 0-1 range
normalized_radius = np.clip(cylinder_radius / max_radius, 0, 1)

# Create realistic background
background = np.full((h, w, 3), (245, 245, 245), dtype=np.uint8)  # Light grey base

# Add subtle gradient to background for depth
gradient_y = np.linspace(0.95, 1.05, h).reshape(-1, 1)
gradient_x = np.linspace(0.98, 1.02, w).reshape(1, -1)
gradient = gradient_y * gradient_x

for i in range(3):
    background[:, :, i] = np.clip(background[:, :, i] * gradient, 0, 255).astype(np.uint8)

# Add very subtle noise to background for texture
noise_bg = np.random.normal(0, 3, (h, w, 3))
background = np.clip(background + noise_bg, 0, 255).astype(np.uint8)

# Create soft shadow for the object
shadow_mask = cv2.dilate(mask, np.ones((15, 15), np.uint8))
shadow_mask = cv2.GaussianBlur(shadow_mask, (25, 25), 8)
shadow_mask = shadow_mask.astype(float) / 255.0

# Apply shadow to background
shadow_color = 0.85  # Darken factor
for i in range(3):
    background[:, :, i] = (background[:, :, i] * (1 - shadow_mask * (1 - shadow_color))).astype(np.uint8)

#Create circular brushed metal texture
# Generate noise
noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)

# Create circular brushing by blurring along circular paths
angles = np.arctan2(dy, dx)

# Create multiple texture layers for realism
base_texture = cv2.GaussianBlur(noise, (3, 3), 0)

# Add radial texture lines (like real brushed metal)
radial_lines = np.zeros((h, w), dtype=np.uint8)
for angle in np.linspace(0, 2*np.pi, 120):  # 120 radial lines
    # Create thin radial lines from center
    line_x = center_x + np.cos(angle) * np.arange(0, max_radius + 50)
    line_y = center_y + np.sin(angle) * np.arange(0, max_radius + 50)

    # Ensure coordinates are within bounds
    valid_idx = (line_x >= 0) & (line_x < w) & (line_y >= 0) & (line_y < h)
    line_x = line_x[valid_idx].astype(int)
    line_y = line_y[valid_idx].astype(int)

    radial_lines[line_y, line_x] = 255

# Blur the radial lines to create brushed effect
radial_texture = cv2.GaussianBlur(radial_lines, (5, 5), 0)

# Combine textures
metal_texture = cv2.addWeighted(base_texture, 0.7, radial_texture, 0.3, 0)

# Apply cylindrical lighting (Lambert's law)
# Assume light source from top-left
light_x, light_y, light_z = -0.5, -0.5, 1.0  # Light direction
light_magnitude = np.sqrt(light_x**2 + light_y**2 + light_z**2)
light_x /= light_magnitude
light_y /= light_magnitude
light_z /= light_magnitude

# Calculate surface normals for cylinder
# For a vertical cylinder, normal vectors point radially outward
normal_x = dx / (max_radius + 1e-6)  # Avoid division by zero
normal_y = np.zeros_like(normal_x)
normal_z = np.sqrt(1 - np.clip(normalized_radius**2, 0, 1))

# Calculate dot product (Lambert's cosine law)
dot_product = normal_x * light_x + normal_y * light_y + normal_z * light_z
lighting = np.clip(dot_product, 0.1, 1.0)  # Clamp to avoid pure black

# Apply lighting to texture
lit_texture = (metal_texture * lighting).astype(np.uint8)

#Add specular highlights
# Specular reflection calculation
view_x, view_y, view_z = 0, 0, 1  # Viewer looking straight down

# Reflection vector: R = 2(N·L)N - L
reflect_x = 2 * dot_product * normal_x - light_x
reflect_y = 2 * dot_product * normal_y - light_y
reflect_z = 2 * dot_product * normal_z - light_z

# Specular intensity
specular_dot = np.clip(reflect_x * view_x + reflect_y * view_y + reflect_z * view_z, 0, 1)
specular = np.power(specular_dot, 50) * 255  # High shininess

#Create final metallic surface
# Convert to BGR
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR)

# Add specular highlights
specular_bgr = cv2.cvtColor(specular.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.addWeighted(metallic, 0.8, specular_bgr, 0.4, 0)

#Add metallic color tint
# Steel/aluminum tint
tint = np.full_like(metallic, (200, 195, 185))  # Cool metallic tone
metallic = cv2.addWeighted(metallic, 0.85, tint, 0.15, 0)

# Add subtle environment reflections on the metallic surface
env_reflection = np.full_like(metallic, (250, 250, 250))  # Light reflection
reflection_intensity = np.power(specular_dot, 20) * 0.2
for i in range(3):
    reflection_mask = (reflection_intensity * (mask / 255.0)).astype(np.float32)
    metallic[:, :, i] = np.clip(metallic[:, :, i] * (1 - reflection_mask) +
                               env_reflection[:, :, i] * reflection_mask, 0, 255).astype(np.uint8)

#Apply mask and enhance edges
# Create the final composite image starting with background
final_image = background.copy()

# Apply metallic texture only where mask is present
mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0
final_image = final_image * (1 - mask_3d) + metallic * mask_3d
final_image = final_image.astype(np.uint8)

# Add edge enhancement for realism
edges = cv2.Canny(mask, 50, 150)
edges_dilated = cv2.dilate(edges, np.ones((2, 2), np.uint8))
edge_highlight = cv2.GaussianBlur(edges_dilated, (7, 7), 2)
edge_highlight_bgr = cv2.cvtColor(edge_highlight, cv2.COLOR_GRAY2BGR)

final_image = cv2.addWeighted(final_image, 1.0, edge_highlight_bgr, 0.3, 0)

#Final contrast and polish
final_image = cv2.convertScaleAbs(final_image, alpha=1.1, beta=5)

# Add ambient occlusion for more realistic depth
ao_mask = cv2.erode(mask, np.ones((8, 8), np.uint8))
ao_mask = cv2.GaussianBlur(ao_mask, (15, 15), 5)
ao_mask = (255 - ao_mask) / 255.0 * 0.3  # Invert and scale

for i in range(3):
    object_pixels = mask > 0
    final_image[object_pixels, i] = np.clip(final_image[object_pixels, i] *
                                          (1 - ao_mask[object_pixels]), 0, 255).astype(np.uint8)

cv2.imwrite("cylindrical_metallic_realistic.png", final_image)
print("Realistic cylindrical metallic render with background saved as cylindrical_metallic_realistic.png")

Realistic cylindrical metallic render with background saved as cylindrical_metallic_realistic.png


In [ ]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

# Find the bounding box and center of the object
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    # Get the largest contour (assuming it's our main object)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
    center_x = x + obj_w // 2
    center_y = y + obj_h // 2
else:
    # Fallback to image center
    center_x, center_y = w // 2, h // 2
    obj_w, obj_h = w, h

# Create coordinate grids
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')

# Creating cylindrical coordinate system
# Calculate distance from center line (for cylindrical effect)
dx = X - center_x
dy = Y - center_y

# For a vertical cylinder, use horizontal distance from center
cylinder_radius = np.abs(dx)
max_radius = obj_w // 2

# Normalize to 0-1 range
normalized_radius = np.clip(cylinder_radius / max_radius, 0, 1)

# Create ultra-realistic background
background = np.full((h, w, 3), (248, 248, 250), dtype=np.uint8)  # Very light grey with slight cool tone

# Create sophisticated lighting gradient for studio-like environment
center_light_x, center_light_y = w * 0.3, h * 0.2  # Light source position
light_falloff = np.sqrt((X - center_light_x)**2 + (Y - center_light_y)**2)
max_distance = np.sqrt(w**2 + h**2)
light_gradient = 1 - (light_falloff / max_distance) * 0.15  # Subtle falloff

# Add multiple gradient layers for depth
gradient_y = np.linspace(0.92, 1.08, h).reshape(-1, 1)  # Top lighter, bottom darker
gradient_x = np.linspace(0.95, 1.05, w).reshape(1, -1)   # Left slightly darker
radial_gradient = 1 + 0.1 * np.exp(-light_falloff / (max_distance * 0.3))  # Radial light

combined_gradient = gradient_y * gradient_x * light_gradient * radial_gradient

for i in range(3):
    background[:, :, i] = np.clip(background[:, :, i] * combined_gradient, 0, 255).astype(np.uint8)

# Add realistic paper/surface texture
texture_noise = np.random.normal(0, 1.5, (h, w))
texture_pattern = np.sin(X * 0.1) * np.sin(Y * 0.1) * 2  # Subtle pattern
total_texture = texture_noise + texture_pattern

for i in range(3):
    background[:, :, i] = np.clip(background[:, :, i] + total_texture, 0, 255).astype(np.uint8)

# Create multiple shadow layers for ultra-realism
# Primary contact shadow (sharp)
contact_shadow = cv2.dilate(mask, np.ones((8, 8), np.uint8))
contact_shadow = cv2.GaussianBlur(contact_shadow, (13, 13), 3)

# Secondary ambient shadow (soft and larger)
ambient_shadow = cv2.dilate(mask, np.ones((25, 25), np.uint8))
ambient_shadow = cv2.GaussianBlur(ambient_shadow, (41, 41), 12)

# Directional shadow (cast shadow)
direction_kernel = np.ones((20, 8), np.uint8)  # Elongated for directional shadow
directional_shadow = cv2.dilate(mask, direction_kernel)
# Shift shadow slightly for light direction
M = np.float32([[1, 0, 5], [0, 1, 3]])  # Translate shadow
directional_shadow = cv2.warpAffine(directional_shadow, M, (w, h))
directional_shadow = cv2.GaussianBlur(directional_shadow, (31, 31), 8)

# Combine shadow layers with different intensities
contact_intensity = 0.25
ambient_intensity = 0.15
directional_intensity = 0.1

total_shadow = np.maximum(
    contact_shadow * contact_intensity,
    np.maximum(ambient_shadow * ambient_intensity, directional_shadow * directional_intensity)
) / 255.0

# Apply shadows to background
for i in range(3):
    background[:, :, i] = (background[:, :, i] * (1 - total_shadow)).astype(np.uint8)

#Create ultra-realistic brushed metal texture
# Generate high-quality noise base
noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)
fine_noise = np.random.randint(0, 256, (h, w), dtype=np.uint8)

# Create multiple texture layers for maximum realism
base_texture = cv2.GaussianBlur(noise, (3, 3), 0)
fine_texture = cv2.GaussianBlur(fine_noise, (1, 1), 0)

# Add concentric circular patterns (like lathe marks)
concentric_lines = np.zeros((h, w), dtype=np.uint8)
for radius in range(5, max_radius + 50, 3):  # Concentric circles every 3 pixels
    circle_mask = np.abs(cylinder_radius - radius) < 1.5
    concentric_lines[circle_mask] = 255

# Blur concentric lines for realistic metal turning marks
concentric_texture = cv2.GaussianBlur(concentric_lines, (3, 1), 0)

# Add radial brushing lines (like real brushed metal)
radial_lines = np.zeros((h, w), dtype=np.uint8)
for angle in np.linspace(0, 2*np.pi, 180):  # More radial lines for finer detail
    line_x = center_x + np.cos(angle) * np.arange(0, max_radius + 50)
    line_y = center_y + np.sin(angle) * np.arange(0, max_radius + 50)

    valid_idx = (line_x >= 0) & (line_x < w) & (line_y >= 0) & (line_y < h)
    line_x = line_x[valid_idx].astype(int)
    line_y = line_y[valid_idx].astype(int)

    radial_lines[line_y, line_x] = 255

# Create anisotropic blur for brushed metal effect
radial_texture = cv2.GaussianBlur(radial_lines, (3, 9), 0)

# Combine all texture layers with realistic weights
metal_texture = cv2.addWeighted(base_texture, 0.4, fine_texture, 0.2, 0)
metal_texture = cv2.addWeighted(metal_texture, 0.7, concentric_texture, 0.15, 0)
metal_texture = cv2.addWeighted(metal_texture, 0.8, radial_texture, 0.2, 0)

# Apply advanced cylindrical lighting with multiple light sources
# Primary light source (key light) from top-left
key_light_x, key_light_y, key_light_z = -0.6, -0.8, 1.2
key_magnitude = np.sqrt(key_light_x**2 + key_light_y**2 + key_light_z**2)
key_light_x /= key_magnitude
key_light_y /= key_magnitude
key_light_z /= key_magnitude

# Secondary fill light from right
fill_light_x, fill_light_y, fill_light_z = 0.4, -0.3, 0.8
fill_magnitude = np.sqrt(fill_light_x**2 + fill_light_y**2 + fill_light_z**2)
fill_light_x /= fill_magnitude
fill_light_y /= fill_magnitude
fill_light_z /= fill_magnitude

# Calculate surface normals for cylinder with better accuracy
normal_x = dx / (max_radius + 1e-6)
normal_y = dy * 0.1 / (max_radius + 1e-6)  # Slight vertical component for realism
normal_z = np.sqrt(1 - np.clip(normalized_radius**2, 0, 0.95))  # Slightly less curved at edges

# Calculate lighting from both sources
key_dot = normal_x * key_light_x + normal_y * key_light_y + normal_z * key_light_z
fill_dot = normal_x * fill_light_x + normal_y * fill_light_y + normal_z * fill_light_z

key_lighting = np.clip(key_dot, 0.05, 1.0)
fill_lighting = np.clip(fill_dot, 0.0, 1.0)

# Combine lighting with realistic weights
total_lighting = np.clip(key_lighting * 0.8 + fill_lighting * 0.3 + 0.15, 0.1, 1.0)

# Apply lighting to texture
lit_texture = (metal_texture * total_lighting).astype(np.uint8)

#Add advanced specular highlights with multiple light sources
# Calculate reflection vectors for both light sources
# Key light reflections
key_reflect_x = 2 * key_dot * normal_x - key_light_x
key_reflect_y = 2 * key_dot * normal_y - key_light_y
key_reflect_z = 2 * key_dot * normal_z - key_light_z

# Fill light reflections
fill_reflect_x = 2 * fill_dot * normal_x - fill_light_x
fill_reflect_y = 2 * fill_dot * normal_y - fill_light_y
fill_reflect_z = 2 * fill_dot * normal_z - fill_light_z

# Viewer direction
view_x, view_y, view_z = 0, 0, 1

# Calculate specular intensities
key_specular_dot = np.clip(key_reflect_x * view_x + key_reflect_y * view_y + key_reflect_z * view_z, 0, 1)
fill_specular_dot = np.clip(fill_reflect_x * view_x + fill_reflect_y * view_y + fill_reflect_z * view_z, 0, 1)

# Different shininess for different types of highlights
sharp_specular = np.power(key_specular_dot, 80) * 255    # Sharp, bright highlights
broad_specular = np.power(key_specular_dot, 25) * 180   # Broader highlights
fill_specular = np.power(fill_specular_dot, 40) * 120   # Fill light specular

# Combine specular components
total_specular = np.clip(sharp_specular + broad_specular * 0.6 + fill_specular * 0.4, 0, 255)

#Create final metallic surface
# Convert to BGR
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR)

# Add specular highlights
specular_bgr = cv2.cvtColor(total_specular.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.addWeighted(metallic, 0.75, specular_bgr, 0.5, 0)

#Add realistic metallic color and material properties
# High-quality steel/aluminum with slight blue tint
base_tint = np.full_like(metallic, (205, 200, 190))  # Cooler metallic tone
warm_tint = np.full_like(metallic, (195, 185, 175))  # Warmer areas
cool_tint = np.full_like(metallic, (215, 210, 205))  # Cooler areas

# Apply different tints based on lighting
warm_areas = total_lighting > 0.7
cool_areas = total_lighting < 0.4

metallic = cv2.addWeighted(metallic, 0.8, base_tint, 0.2, 0)
metallic[warm_areas] = cv2.addWeighted(metallic, 0.85, warm_tint, 0.15, 0)[warm_areas]
metallic[cool_areas] = cv2.addWeighted(metallic, 0.85, cool_tint, 0.15, 0)[cool_areas]

# Add realistic environment reflections with color variation
env_reflection_bright = np.full_like(metallic, (255, 255, 255))  # Bright highlights
env_reflection_mid = np.full_like(metallic, (235, 235, 240))     # Mid-tone reflections
env_reflection_shadow = np.full_like(metallic, (180, 180, 185))  # Shadow reflections

# Create complex reflection patterns
bright_reflection_intensity = np.power(key_specular_dot, 15) * 0.3
mid_reflection_intensity = np.power(key_specular_dot, 8) * 0.2
shadow_reflection_intensity = (1 - total_lighting) * 0.1

for i in range(3):
    bright_mask = (bright_reflection_intensity * (mask / 255.0)).astype(np.float32)
    mid_mask = (mid_reflection_intensity * (mask / 255.0)).astype(np.float32)
    shadow_mask = (shadow_reflection_intensity * (mask / 255.0)).astype(np.float32)

    metallic[:, :, i] = np.clip(
        metallic[:, :, i] * (1 - bright_mask - mid_mask - shadow_mask) +
        env_reflection_bright[:, :, i] * bright_mask +
        env_reflection_mid[:, :, i] * mid_mask +
        env_reflection_shadow[:, :, i] * shadow_mask, 0, 255
    ).astype(np.uint8)

#Apply mask and enhance edges
# Create the final composite image starting with background
final_image = background.copy()

# Apply metallic texture only where mask is present
mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0
final_image = final_image * (1 - mask_3d) + metallic * mask_3d
final_image = final_image.astype(np.uint8)

# Add sophisticated edge enhancement for ultra-realism
edges = cv2.Canny(mask, 30, 120)

# Create multiple edge effects
sharp_edges = cv2.dilate(edges, np.ones((1, 1), np.uint8))
soft_edges = cv2.dilate(edges, np.ones((3, 3), np.uint8))

# Apply different blurs for different edge effects
sharp_edge_highlight = cv2.GaussianBlur(sharp_edges, (3, 3), 1)
soft_edge_highlight = cv2.GaussianBlur(soft_edges, (9, 9), 3)
rim_light = cv2.GaussianBlur(soft_edges, (15, 15), 6)

# Convert to BGR
sharp_edge_bgr = cv2.cvtColor(sharp_edge_highlight, cv2.COLOR_GRAY2BGR)
soft_edge_bgr = cv2.cvtColor(soft_edge_highlight, cv2.COLOR_GRAY2BGR)
rim_light_bgr = cv2.cvtColor(rim_light, cv2.COLOR_GRAY2BGR)

# Apply edge enhancements with different intensities
final_image = cv2.addWeighted(final_image, 1.0, sharp_edge_bgr, 0.4, 0)
final_image = cv2.addWeighted(final_image, 1.0, soft_edge_bgr, 0.2, 0)
final_image = cv2.addWeighted(final_image, 1.0, rim_light_bgr, 0.15, 0)

#Final ultra-realistic processing
# Apply sophisticated tone mapping
final_image = cv2.convertScaleAbs(final_image, alpha=1.15, beta=8)

# Add advanced ambient occlusion with multiple scales
# Fine ambient occlusion
fine_ao = cv2.erode(mask, np.ones((4, 4), np.uint8))
fine_ao = cv2.GaussianBlur(fine_ao, (9, 9), 2)

# Broad ambient occlusion
broad_ao = cv2.erode(mask, np.ones((12, 12), np.uint8))
broad_ao = cv2.GaussianBlur(broad_ao, (21, 21), 8)

# Contact ambient occlusion (very tight)
contact_ao = cv2.erode(mask, np.ones((2, 2), np.uint8))
contact_ao = cv2.GaussianBlur(contact_ao, (5, 5), 1)

# Combine AO layers
fine_ao_factor = (255 - fine_ao) / 255.0 * 0.2
broad_ao_factor = (255 - broad_ao) / 255.0 * 0.15
contact_ao_factor = (255 - contact_ao) / 255.0 * 0.3

total_ao = np.clip(fine_ao_factor + broad_ao_factor + contact_ao_factor, 0, 0.6)

# Apply ambient occlusion
for i in range(3):
    object_pixels = mask > 0
    final_image[object_pixels, i] = np.clip(final_image[object_pixels, i] *
                                          (1 - total_ao[object_pixels]), 0, 255).astype(np.uint8)

# Add final color grading for photorealism
# Slight color temperature adjustment
final_image[:, :, 0] = np.clip(final_image[:, :, 0] * 0.98, 0, 255)  # Reduce blue slightly
final_image[:, :, 2] = np.clip(final_image[:, :, 2] * 1.02, 0, 255)  # Increase red slightly

# Add final micro-contrast enhancement
clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8,8))
for i in range(3):
    final_image[:, :, i] = clahe.apply(final_image[:, :, i])

cv2.imwrite("ultra_realistic_metallic.png", final_image)
print("Ultra-realistic metallic render with studio background saved as ultra_realistic_metallic.png")

Ultra-realistic metallic render with studio background saved as ultra_realistic_metallic.png


In [ ]:
import cv2
import numpy as np

# Load mask
mask = cv2.imread("15.png_mask.png", cv2.IMREAD_GRAYSCALE)
_, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

h, w = mask.shape

# Find the bounding box and center of the object
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, obj_w, obj_h = cv2.boundingRect(largest_contour)
    center_x = x + obj_w // 2
    center_y = y + obj_h // 2
else:
    center_x, center_y = w // 2, h // 2
    obj_w, obj_h = w, h

# Create coordinate grids
Y, X = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
dx = X - center_x
dy = Y - center_y

# For cylindrical geometry
cylinder_radius = np.abs(dx)
max_radius = obj_w // 2
normalized_radius = np.clip(cylinder_radius / max_radius, 0, 1)

# Create professional studio background (like reference image)
background = np.full((h, w, 3), (252, 252, 254), dtype=np.uint8)  # Very light, almost white

# Studio lighting gradient (bright center, subtle falloff)
studio_center_x, studio_center_y = w * 0.5, h * 0.3
studio_distance = np.sqrt((X - studio_center_x)**2 + (Y - studio_center_y)**2)
max_studio_distance = np.sqrt(w**2 + h**2)
studio_gradient = 1 - (studio_distance / max_studio_distance) * 0.08

# Apply very subtle gradient
for i in range(3):
    background[:, :, i] = np.clip(background[:, :, i] * studio_gradient, 240, 255).astype(np.uint8)

# Create ultra-precise shadows matching reference
# Very soft, minimal shadow like in professional photography
soft_shadow = cv2.dilate(mask, np.ones((12, 12), np.uint8))
soft_shadow = cv2.GaussianBlur(soft_shadow, (25, 25), 10)
soft_shadow = soft_shadow.astype(float) / 255.0

# Apply very subtle shadow (barely visible, like in reference)
shadow_intensity = 0.08  # Very light shadow
for i in range(3):
    background[:, :, i] = (background[:, :, i] * (1 - soft_shadow * shadow_intensity)).astype(np.uint8)

# Create highly polished chrome-like metal texture
# Minimal base texture for ultra-smooth surface
smooth_noise = np.random.normal(0, 8, (h, w))
base_texture = cv2.GaussianBlur(smooth_noise.astype(np.uint8), (1, 1), 0)

# Add very fine machining marks (barely visible)
fine_lines = np.zeros((h, w), dtype=np.float32)
for angle in np.linspace(0, 2*np.pi, 60):
    line_x = center_x + np.cos(angle) * np.arange(0, max_radius + 20)
    line_y = center_y + np.sin(angle) * np.arange(0, max_radius + 20)

    valid_idx = (line_x >= 0) & (line_x < w) & (line_y >= 0) & (line_y < h)
    if len(valid_idx) > 0:
        line_x = line_x[valid_idx].astype(int)
        line_y = line_y[valid_idx].astype(int)
        fine_lines[line_y, line_x] = 30  # Very subtle

# Blur for smooth finish
fine_texture = cv2.GaussianBlur(fine_lines, (3, 3), 0)

# Combine textures (mostly smooth with minimal texture)
metal_texture = cv2.addWeighted(base_texture.astype(np.float32), 0.3, fine_texture, 0.1, 128)
metal_texture = np.clip(metal_texture, 0, 255).astype(np.uint8)

# Advanced lighting setup (like studio photography)
# Key light from upper left (strong)
key_light_x, key_light_y, key_light_z = -0.4, -0.7, 1.0
key_magnitude = np.sqrt(key_light_x**2 + key_light_y**2 + key_light_z**2)
key_light_x /= key_magnitude
key_light_y /= key_magnitude
key_light_z /= key_magnitude

# Fill light from right (softer)
fill_light_x, fill_light_y, fill_light_z = 0.6, -0.3, 0.8
fill_magnitude = np.sqrt(fill_light_x**2 + fill_light_y**2 + fill_light_z**2)
fill_light_x /= fill_magnitude
fill_light_y /= fill_magnitude
fill_light_z /= fill_magnitude

# Calculate precise surface normals for smooth cylinder
normal_x = dx / (max_radius + 1e-6)
normal_y = np.zeros_like(normal_x)
normal_z = np.sqrt(np.clip(1 - normalized_radius**2, 0.1, 1.0))

# Calculate lighting from both sources
key_dot = np.clip(normal_x * key_light_x + normal_y * key_light_y + normal_z * key_light_z, 0, 1)
fill_dot = np.clip(normal_x * fill_light_x + normal_y * fill_light_y + normal_z * fill_light_z, 0, 1)

# Combine lighting with studio-like setup
total_lighting = np.clip(key_dot * 0.7 + fill_dot * 0.4 + 0.2, 0.2, 1.0)

# Apply lighting
lit_texture = (metal_texture * total_lighting).astype(np.uint8)

# Create ultra-sharp specular highlights (like chrome)
view_x, view_y, view_z = 0, 0, 1

# Key light reflections
key_reflect_x = 2 * key_dot * normal_x - key_light_x
key_reflect_y = 2 * key_dot * normal_y - key_light_y
key_reflect_z = 2 * key_dot * normal_z - key_light_z

# Fill light reflections
fill_reflect_x = 2 * fill_dot * normal_x - fill_light_x
fill_reflect_y = 2 * fill_dot * normal_y - fill_light_y
fill_reflect_z = 2 * fill_dot * normal_z - fill_light_z

# Calculate specular intensities
key_specular_dot = np.clip(key_reflect_x * view_x + key_reflect_y * view_y + key_reflect_z * view_z, 0, 1)
fill_specular_dot = np.clip(fill_reflect_x * view_x + fill_reflect_y * view_y + fill_reflect_z * view_z, 0, 1)

# Ultra-high shininess for chrome-like finish
chrome_specular = np.power(key_specular_dot, 150) * 255  # Very sharp highlights
broad_specular = np.power(key_specular_dot, 40) * 180   # Broader chrome reflections
fill_specular = np.power(fill_specular_dot, 80) * 120   # Fill light specular

total_specular = np.clip(chrome_specular + broad_specular * 0.4 + fill_specular * 0.3, 0, 255)

# Create base metallic surface
metallic = cv2.cvtColor(lit_texture, cv2.COLOR_GRAY2BGR)

# Add specular highlights
specular_bgr = cv2.cvtColor(total_specular.astype(np.uint8), cv2.COLOR_GRAY2BGR)
metallic = cv2.addWeighted(metallic, 0.7, specular_bgr, 0.6, 0)

# Apply chrome-like color (cooler, more reflective)
chrome_tint = np.full_like(metallic, (220, 215, 205))  # Cool chrome tone
metallic = cv2.addWeighted(metallic, 0.8, chrome_tint, 0.2, 0)

# Add environment reflections (bright studio lights and surfaces)
# Bright reflections (studio lights)
bright_env = np.full_like(metallic, (255, 255, 255))
bright_reflection = np.power(key_specular_dot, 25) * 0.4

# Mid-tone reflections (studio walls/ceiling)
mid_env = np.full_like(metallic, (240, 245, 250))
mid_reflection = np.power(key_specular_dot, 12) * 0.25

# Apply environment reflections
for i in range(3):
    bright_mask = (bright_reflection * (mask / 255.0)).astype(np.float32)
    mid_mask = (mid_reflection * (mask / 255.0)).astype(np.float32)

    metallic[:, :, i] = np.clip(
        metallic[:, :, i] * (1 - bright_mask - mid_mask) +
        bright_env[:, :, i] * bright_mask +
        mid_env[:, :, i] * mid_mask, 0, 255
    ).astype(np.uint8)

# Composite final image
final_image = background.copy()
mask_3d = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR) / 255.0
final_image = final_image * (1 - mask_3d) + metallic * mask_3d
final_image = final_image.astype(np.uint8)

# Add ultra-precise edge definition
edges = cv2.Canny(mask, 40, 160)
# Sharp edge highlights for chrome finish
edge_highlight = cv2.GaussianBlur(edges, (3, 3), 0.5)
edge_highlight_bgr = cv2.cvtColor(edge_highlight, cv2.COLOR_GRAY2BGR)
final_image = cv2.addWeighted(final_image, 1.0, edge_highlight_bgr, 0.5, 0)

# Final chrome-like processing
# High contrast for sharp reflections
final_image = cv2.convertScaleAbs(final_image, alpha=1.25, beta=5)

# Minimal ambient occlusion (chrome has very little self-shadowing)
minimal_ao = cv2.erode(mask, np.ones((3, 3), np.uint8))
minimal_ao = cv2.GaussianBlur(minimal_ao, (7, 7), 2)
ao_factor = (255 - minimal_ao) / 255.0 * 0.1  # Very subtle AO

for i in range(3):
    object_pixels = mask > 0
    final_image[object_pixels, i] = np.clip(final_image[object_pixels, i] *
                                          (1 - ao_factor[object_pixels]), 0, 255).astype(np.uint8)

# Add final chrome polish effect
# Enhance saturation slightly in highlights
hsv = cv2.cvtColor(final_image, cv2.COLOR_BGR2HSV)
hsv[:, :, 1] = np.clip(hsv[:, :, 1] * 0.7, 0, 255)  # Reduce saturation for chrome look
hsv[:, :, 2] = np.clip(hsv[:, :, 2] * 1.1, 0, 255)  # Increase brightness
final_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

cv2.imwrite("chrome_realistic_metallic.png", final_image)
print("Ultra-realistic chrome metallic render saved as chrome_realistic_metallic.png")

Ultra-realistic chrome metallic render saved as chrome_realistic_metallic.png
